In [1]:
import thermo
from thermo.chemical import Chemical

In [2]:
import math
import fluids
from fluids.units import *


Given:  Water at 60 degF discharges from a tank with 22-fet average head to atmosphere through:
        200 feet - 3" Schedule 40 pipe
        6 - 3" standard 90 deg threaded elbows
        1 - 3" flanged ball valve having a 2-3/8" diameter seat, 16 deg conical inlet, and 30 deg conical outlet end.  Sharp-edged entrance is flush with the inside of the tank.
        
Find: Velocity of flow in the pipe and rate of discharge in gallons per minute.

In [3]:
T = (60*u.degF).to(u.degK)

Assume atmospheric pressures

In [4]:
P = (1*u.atm).to(u.Pa)

Get the water properties from PubChem

In [5]:
water = Chemical('water', P=P.magnitude, T=T.magnitude)

# Solve using the fluids-built in functions exclusively

Only the following types of clean, new pipe have data available:
‘Plastic coated’
‘Carbon steel, honed bare’
‘Cr13, electropolished bare’
‘Cement lining’
‘Carbon steel, bare’
‘Fiberglass lining’
‘Cr13, bare’

In [6]:
mu = water.mu*u.Pa*u.sec
rho = water.rho*u.kilogram/u.meter**3

d1 = (2+3/8)*u.inch
d2 = nearest_pipe(NPS=3*u.inch, schedule='40')[1].to(u.inch)
beta = d1/d2

# Sum the Ks

In [7]:
K_inlet = 0.5 #Crane A-30
K_exit = 1.0 #Crane A-30


#Re for fully turbulent flow
Re = 10**8

e = roughness_Farshad('Carbon steel, bare', D=d2.to(u.meter))
f = friction_factor(Re=Re, eD=e/d2.to(u.meter))

sin_theta_div2_inlet = math.sin(8*math.pi/180)
sin_theta_div2_outlet = math.sin(15*math.pi/180)

ft = 0.017 #To use the formulas for K2 from Crane 410, K1 is calculated using friciton at complete turbulence
            #need to get the ft from A-27.  The difference in using f as calculated for the remainder
            #is not large.
K1 = 3*ft
#Since inlet and outlet angles differ, Formula 5 from Crane A-29 needs to be modified
K2 = (K1 + 0.80*sin_theta_div2_inlet*(1-beta) + 2.6*sin_theta_div2_outlet*(1-beta**2)**2)/beta**4
K_valve = K2

#Elbows
n_elbows = 6
K_elbows = (30*f) * n_elbows

#Pipe
D = d2
L=200*u.feet
K_pipe= f*L/D.to(u.feet)

K_tot = K_inlet+K_exit+K_valve+K_elbows+K_pipe

In [8]:
#velocity of flow in the pipe
g=32.2*u.feet/u.sec**2
hl = 22*u.feet
v = math.sqrt((2*g*hl/K_tot).magnitude)*u.feet/u.sec  #math module doesn't handle units

#flow in pipe
Q =(2.448*v*d2**2).magnitude*u.gallons/u.minute

print(" Pipe flow velocity is: {}\n Flow rate is: {}".format(v,Q))

 Pipe flow velocity is: 8.941725633644984 foot / second
 Flow rate is: 205.99809676169716 gallon / minute


In [9]:
#Calculate the RE number to make sure that we have realistic fricion factor
V = v.to(u.meter/u.sec)
Re = Reynolds(D=d2.to(u.meter), rho=rho, V=v, mu=mu)
Re

<Quantity(187292.6611424989, 'dimensionless')>

Reading off Crane A-26, with the Re=1.8x10^5 places us in the transistion zone, and f is ~0.02.  Crane says for this small difference we can forgo any correctoin of K for the pipe.
The correction is to use the f=0.02 the above calculations.
Great thing about have this programmed is that we can interate.  If we set the Re=1.76x10^5 for the friciton factor calculation we also get the flow conditions we need to give us back the same Re.

# Solution as Provided by Crane 410

In [10]:
f = 0.017 #Crane A-27

K1 = 3*f
#Since inlet and outlet angles differ, Formula 5 from Crane A-29 needs to be modified
K2 = (K1 + 0.80*sin_theta_div2_inlet*(1-beta) + 2.6*sin_theta_div2_outlet*(1-beta**2)**2)/beta**4


#Elbows
n_elbows = 6
K_elbows = (30*f) * n_elbows

#Pipe
D = d2
L=200*u.feet
K_pipe= f*L/D.to(u.feet)

K_tot = K_inlet+K_exit+K_valve+K_elbows+K_pipe

#velocity of flow in the pipe
g=32.2*u.feet/u.sec**2
hl = 22*u.feet
v = math.sqrt((2*g*hl/K_tot).magnitude)*u.feet/u.sec  #math module doesn't handle units

#flow in pipe
Q =(2.448*v*d2**2).magnitude*u.gallons/u.minute

print(" Pipe flow velocity is: {}\n Flow rate is: {}".format(v,Q))

#Calculate Reynolds number to verify that friciton factor of 0.017 is correct fo flow conditions:
Rho = rho.to(u.lb/u.feet**3)
Mu = mu.to(u.cP) 
Re = (50.66*Q*Rho/(d2*Mu)).magnitude
print(Re)

 Pipe flow velocity is: 8.781555746535433 foot / second
 Flow rate is: 202.30812759299954 gallon / minute
183940.7488175578


Note from Crane solution:  f for 3" pipe is less than 0.02. Therefore, flow is in the transition zone (slightly less than fully turbulent) bu the difference is small enough to forego any correction of K for the pipe.


The calculation from the fluids module vs what Crane 410 gives is pretty close.